In [ ]:
import requests

In [101]:
import pandas as pd
from pandas import json_normalize

info_params = {
    'id': '1077894',
}

info_url = 'https://www.fotmob.com/api/playerData'

# Load data using json normalize
# The point of the method and extra code is to ensure that the data is loaded consistently every time
def load_player_info(url , params):
    with requests.Session() as session:
        try:
            # Get the player information
            response = session.get(info_url , params = params)
            response.raise_for_status()
            data = response.json()

            # Extract player information
            player_info = data.get('playerInformation' , {})
            info_df = json_normalize(player_info)

            # Extract player name
            player_name = data.get('name')
            return info_df , player_name  # Return both DataFrame and player name as a tuple
        except requests.exceptions.RequestException as e:
            print(f"Error loading data: {e}")
    return pd.DataFrame() , None  # Return empty DataFrame and None for player name if there's an error

# Call load_data method and drop all NaN values for columns that meet the threshold
info_df , player_name = load_player_info(info_url , info_params)
info_df = info_df.dropna(axis = 1 , thresh = len(info_df) * 0.9)

# Rename columns and drop translationKey column
info_df = info_df.rename(columns={'title' : 'Title' , 'value.fallback' : 'Info'})
info_df = info_df.drop(columns = ['translationKey'])

# Add player's name as a new row
info_df.loc[len(info_df)] = ['Name' , player_name]

file_name = f"{player_name.replace(' ', '_')}_Player_Info.csv"
info_df.to_csv(file_name , index = False , encoding = 'utf-8-sig')

info_df

,Title,Info
0,Height,186 cm
1,Shirt,5
2,Age,20
3,Preferred foot,Right
4,Country,England
5,Market value,€115M
6,Name,Jude Bellingham


In [102]:
# change url and params due to the data being in a different api
stats_params = {
    'playerId': '1077894',
    'seasonId': '2023/2024-87',
}

stats_url = 'https://www.fotmob.com/api/playerStats'

# repeat the method used to get the player data
def load_stats(url , params):
    with requests.Session() as session:
        try:
            response = session.get(url , params = params)
            response.raise_for_status()
            data = response.json()

            # since the data is nested multiple times, we will run a for loop to iterate over all of the nested keys
            player_stats = data.get('statsSection' , {}).get('items' , [])
            all_stats = []
            for item in player_stats:
                inner_key = item.get('items' , [])
                for item in inner_key:
                    all_stats.append(item)

            return stats_df
        except requests.exceptions.RequestException as e:
            print(f"Error loading data: {e}")
    return pd.DataFrame()

stats_df = load_stats(stats_url , stats_params)
stats_df = stats_df.rename(columns = {'title' : 'Title' , 'localizedTitleId' : 'Text' , 'statValue' : 'Stat' , 'per90' : 'statPer90' , 'percentileRank' : 'Percentile' , 'percentileRankPer90' : 'percentilePer90'})

file_name = f"{player_name.replace(' ', '_')}_Player_Stats.csv"
stats_df.to_csv(file_name , index = False , encoding = 'utf-8-sig')

stats_df

,Title,Text,Stat,statPer90,Percentile,percentilePer90
0,Goals,goals,19,0.736117,100.000000,95.890411
1,xG,expected_goals,11.12,0.430702,100.000000,89.041096
2,xGOT,expected_goals_on_target,13.65,0.528660,100.000000,94.520548
3,Penalty goals,goals_subtitle,1,0.038743,87.671233,90.410959
4,xG excl. penalty,non_penalty_xg,10.33,0.400157,100.000000,87.671233
5,Shots,shots,63,2.440809,90.410959,69.863014
6,Shots on target,ShotsOnTarget,35,1.356005,97.260274,89.041096
7,Assists,assists,6,0.232458,91.780822,80.821918
8,xA,expected_assists,4.31,0.166835,84.931507,67.123288
9,Accurate passes,successful_passes,1347,52.186827,100.000000,97.260274


In [103]:
stats_params = {
    'playerId': '1077894',
    'seasonId': '2023/2024-87',
}

stats_url = 'https://www.fotmob.com/api/playerStats'

# repeat the method used to get the player data
def load_shotmap(url, params):
    with requests.Session() as session:
        try:
            response = session.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            shotmap = data.get('shotmap' , {})
            shots_df = json_normalize(shotmap)

            return shots_df
        except requests.exceptions.RequestException as e:
            print(f"Error loading data: {e}")
    return pd.DataFrame()

shots_df = load_shotmap(stats_url, stats_params)
shots_df = shots_df.drop(columns = ['goalCrossedY' , 'goalCrossedZ' , 'homeTeamId' , 'homeScore' , 'awayTeamId' , 'id' , 'playerId' , 'playerName' , 'teamColor' , 'teamColorDark' , 'teamId'])
shots_df = shots_df.drop(columns = ['onGoalShot.zoomRatio' , 'minAdded' , 'awayScore'])
shots_df = shots_df.drop(columns = ['box'])
shots_df = shots_df.drop(columns = ['isSavedOffLine'])
shots_df = shots_df.drop(columns = ['matchId'])

shots_df = shots_df.rename(columns = {
    'eventType' : 'Type' ,
    'min' : 'Minute' ,
    'expectedGoals' : 'xG' ,
    'expectedGoalsOnTarget' : 'onTargetxG' ,
    'isFromInsideBox' : 'insideBox' ,
    'onGoalShot.x' : 'onGoalShotX' ,
    'onGoalShot.y' : 'onGoalShotY' ,
    'period' : 'Half'
})

file_name = f"{player_name.replace(' ', '_')}_Player_Shots.csv"
shots_df.to_csv(file_name , index = False , encoding = 'utf-8-sig')

shots_df

,Type,x,y,Minute,isBlocked,isOnTarget,xG,onTargetxG,shotType,situation,Half,isOwnGoal,insideBox,homeTeamName,awayTeamName,matchDate,onGoalShotX,onGoalShotY,blockedX,blockedY
0,Goal,92.400000,28.539048,36,False,True,0.078295,0.4745,RightFoot,FromCorner,FirstHalf,False,True,Athletic Club,Real Madrid,2023-08-12T19:30:00Z,0.152778,0.300668,NaN,NaN
1,AttemptSaved,98.245614,37.355000,36,True,True,0.150605,NaN,RightFoot,RegularPlay,FirstHalf,False,True,Athletic Club,Real Madrid,2023-08-12T19:30:00Z,0.939484,0.322751,99.403509,37.278750
2,AttemptSaved,92.400000,40.973810,5,True,True,0.058295,NaN,LeftFoot,RegularPlay,FirstHalf,False,True,Almeria,Real Madrid,2023-08-19T17:30:00Z,0.556217,0.322751,93.700000,40.301429
3,AttemptSaved,91.500000,33.313750,10,False,True,0.028291,0.1421,OtherBodyParts,RegularPlay,FirstHalf,False,True,Almeria,Real Madrid,2023-08-19T17:30:00Z,0.737765,0.536786,104.715517,33.847500
4,Goal,98.631579,36.440000,19,False,True,0.175397,0.9757,RightFoot,RegularPlay,FirstHalf,False,True,Almeria,Real Madrid,2023-08-19T17:30:00Z,1.887566,0.074742,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Goal,100.258621,45.121783,10,False,True,0.039649,0.6485,LeftFoot,RegularPlay,FirstHalf,False,True,Real Madrid,Deportivo Alaves,2024-05-14T19:30:00Z,1.867394,0.139293,NaN,NaN
59,AttemptSaved,88.500000,48.695032,45,False,True,0.017290,0.0226,RightFoot,RegularPlay,FirstHalf,False,False,Real Madrid,Deportivo Alaves,2024-05-14T19:30:00Z,1.060516,0.332943,103.198276,35.525000
60,Miss,93.900000,31.102500,63,False,False,0.155939,NaN,RightFoot,RegularPlay,SecondHalf,False,True,Real Madrid,Deportivo Alaves,2024-05-14T19:30:00Z,2.000000,0.086967,NaN,NaN
61,AttemptSaved,87.211765,37.732000,22,True,True,0.079918,NaN,RightFoot,RegularPlay,FirstHalf,False,False,Real Madrid,Real Betis,2024-05-25T19:00:00Z,0.959656,0.322751,91.000000,37.278750
